# 1. Observations on 1 pixel kernel

# 2. BMENet with maxpool

### 2.1 Train loss curve

### 2.2 Confusion Matrix

# 3. BMENet with stride

### 3.1 Train loss curve

### 3.2 Confusion Matrix

# 4. Table1: Overall accuracy of 2 models table

# 5. Table2: Per class accuracy of 2 models (10x2 table)

# 6. Observations of Maxpool vs Stride

# 7. Skip Connections with MSCOCO

### 7.1 5x3 images

### 7.2 train loss curve

### 7.3 conf matrix 5x5

### 7.4 overall accuracy

### 7.5 per class accuracy 5x1 table

### 7.6 observations